# PyCity Schools Analysis

Analysis:

The study comprises the school data for the entire district, in which variables such as school type, student population and per-student spending were employed to parse the data against the metrics of passing grades for math, reading and combined, in order to find trends that explain the performance in those metrics and conclude which are determinant of a better or worse performance.

Key Findings:
 - As a whole, reading passing has faired better than math, both in average scores and percentage of students passing, which suggests more emphasis needs to be put in preparing students in math. The school with the lowest average math score in fact have low math scores in all four grades; this can help in identifying the schools were more immediate action is neeeded. 

 - Charter schools outperformed district schools in overall passing rate. All top 5 schools were charter; in contrast, all bottom 5 schools were disctrict. This suggest emphasis needs to be put in supporting district schools.

 - surprisingly, schools with lower per student budget performed better than schools with higher per student budget. These schools tend to be charter schools, which in turn tend to be small in size. This suggests that the methodology of teaching and the student population (which could be a proxy for teacher:student ratio, although this is not shown in the data and it is thus an assumption) can be a better determinant of student success rather than budget allocation per se.

# ------------------------------------------------------------------------

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_input_path = "Resources/schools_complete.csv"
student_input_path = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_input_path)
student_data = pd.read_csv(student_input_path)

# Visualizing the structure of school data
school_data.head()



,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [2]:
#Visualizing the structure of student data
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [3]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Visualizing combined dataframe
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [4]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["school_name"].unique())
school_count

15

In [5]:
# Calculate the total number of students
student_count = school_data_complete["student_name"].value_counts().sum()
student_count

39170

In [6]:
# Calculate the total budget
total_budget = school_data["budget"].sum()
total_budget

24649428

In [7]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [8]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [9]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [10]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [11]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [12]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schoos":[school_count],
                                "Total Students":[student_count],
                                "Total Budget":[total_budget],
                                "Average Math Score":[average_math_score],
                                "average_reading_score":[average_reading_score],
                                "% Passing Math": [passing_math_percentage],
                                "% Passing Reading": [passing_reading_percentage],
                                "% Overall Passing": [overall_passing_rate],
                                })

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schoos,Total Students,Total Budget,Average Math Score,average_reading_score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [13]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]

#Visualizing
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [14]:
# Calculate the total student count
    # per_school_group = school_data_complete.groupby(["school_name"])
    # per_school_counts = per_school_group["student_name"].count()
    # per_school_counts

per_school_counts = school_data_complete.groupby(["school_name"]).count()["student_name"]

#Visualizing
per_school_counts

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: student_name, dtype: int64

In [15]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean(numeric_only = True)["budget"]
per_school_capita = per_school_budget / per_school_counts

#Visualizing
per_school_budget

school_name
Bailey High School       3124928.0
Cabrera High School      1081356.0
Figueroa High School     1884411.0
Ford High School         1763916.0
Griffin High School       917500.0
Hernandez High School    3022020.0
Holden High School        248087.0
Huang High School        1910635.0
Johnson High School      3094650.0
Pena High School          585858.0
Rodriguez High School    2547363.0
Shelton High School      1056600.0
Thomas High School       1043130.0
Wilson High School       1319574.0
Wright High School       1049400.0
Name: budget, dtype: float64

In [16]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby(["school_name"]).mean(numeric_only = True)["math_score"]
per_school_reading = school_data_complete.groupby(["school_name"]).mean(numeric_only = True)["reading_score"]


In [17]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete[(school_data_complete["math_score"]>70)]

In [18]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[(school_data_complete["reading_score"]>70)]

In [19]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
# print(len(passing_math_and_reading["school_name"].unique()))

In [20]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [21]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.concat({"School Type":school_types,
                                "Total Students":per_school_counts,
                                "Total School Budget":per_school_budget,
                                "Per Student Budget":per_school_capita, 
                                "Average Math Score":per_school_math,
                                "Average Reading Score":per_school_reading,
                                "%Passing Math":per_school_passing_math,
                                "%Passing Reading":per_school_passing_reading,
                                "%Overall Passing":overall_passing_rate}, 
                                axis =1)

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Sorting alphabetically by school name
per_school_summary = per_school_summary.sort_values("school_name")

# Since the previous line of code has "School_name" as the index, we reset the index so that it is an integer starting at 0

per_school_summary = per_school_summary.reset_index() 

# # Display the DataFrame
per_school_summary

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,54.642283
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.334769
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,53.204476
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,54.289887
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,90.599455
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,53.527508
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,89.227166
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,53.513884
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,53.539172
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [22]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools_df = per_school_summary.sort_values("%Overall Passing", ascending=False)
top_schools_df.head()



,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.334769
12,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,90.214067,92.905199,90.948012
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,90.599455
13,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,90.582567
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [23]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools_df = per_school_summary.sort_values("%Overall Passing")
bottom_schools_df.head()

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
10,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,52.988247
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,53.204476
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,53.513884
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,53.527508
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,53.539172


## Math Scores by Grade

In [24]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby("school_name").mean(numeric_only = True)
tenth_graders_scores = tenth_graders.groupby("school_name").mean(numeric_only = True)
eleventh_graders_scores = eleventh_graders.groupby("school_name").mean(numeric_only = True)
twelfth_graders_scores = twelfth_graders.groupby("school_name").mean(numeric_only = True)

# # Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.concat({"9th Grade":ninth_grade_math_scores,
                                    "10th Grade":tenth_grader_math_scores,
                                    "11th Grade": eleventh_grader_math_scores,
                                    "12th Grade": twelfth_grader_math_scores,},
                                    axis=1)

# # Minor data wrangling
math_scores_by_grade.index.name = None

# # Display the DataFrame
math_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [25]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby("school_name").mean(numeric_only = True)
tenth_graders_scores = tenth_graders.groupby("school_name").mean(numeric_only = True)
eleventh_graders_scores = eleventh_graders.groupby("school_name").mean(numeric_only = True)
twelfth_graders_scores = twelfth_graders.groupby("school_name").mean(numeric_only = True)

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.concat({"9th Grade":ninth_grade_reading_scores,
                                    "10th Grade":tenth_grader_reading_scores,
                                    "11th Grade": eleventh_grader_reading_scores,
                                    "12th Grade": twelfth_grader_reading_scores,},
                                    axis=1)

# Minor data wrangling
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [26]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [27]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

# Fixing format of "Per Student Budget" so it can be treated as a float and thus the cut method can work
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace("$","").astype("float64")


/var/folders/v7/61jxc77x4_12lwrhn3fh50yr0000gn/T/ipykernel_73285/4140915297.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace("$","").astype("float64")


In [28]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"],bins = spending_bins,labels=labels)

# Visualizing the df
school_spending_df

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing,Spending Ranges (Per Student)
0,Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,64.630225,79.300643,54.642283,$585-630
1,Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,89.558665,93.864370,91.334769,<$585
2,Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,63.750424,78.433367,53.204476,$630-645
3,Ford High School,District,2739,"$1,763,916.00",644.0,77.102592,80.746258,65.753925,77.510040,54.289887,$630-645
4,Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,89.713896,93.392371,90.599455,$585-630
5,Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,64.746494,78.187702,53.527508,$645-680
6,Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,90.632319,92.740047,89.227166,<$585
7,Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,63.318478,78.813850,53.513884,$645-680
8,Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.966394,63.852132,78.281874,53.539172,$645-680
9,Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,91.683992,92.203742,90.540541,$585-630


In [29]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean(numeric_only = True)["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean(numeric_only = True)["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean(numeric_only = True)["%Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean(numeric_only = True)["%Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean(numeric_only = True)["%Overall Passing"]

In [30]:
# Assemble into DataFrame
spending_summary = pd.concat([spending_math_scores, 
                                spending_reading_scores,
                                spending_passing_math, 
                                spending_passing_reading, 
                                overall_passing_spending], 
                                axis=1)
# Display results

spending_summary

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,90.350436,93.325838,90.369459
$585-630,81.899826,83.155286,83.980055,89.378647,81.418596
$630-645,78.518855,81.624473,70.946108,81.648261,62.857656
$645-680,76.997210,81.027843,63.972368,78.427809,53.526855


## Scores by School Size

In [31]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [32]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"],bins=size_bins, labels=labels)

#Visualizing
per_school_summary

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing,School Size
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,54.642283,Large (2000-5000)
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.334769,Medium (1000-2000)
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,53.204476,Large (2000-5000)
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,54.289887,Large (2000-5000)
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,90.599455,Medium (1000-2000)
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,53.527508,Large (2000-5000)
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,89.227166,Small (<1000)
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,53.513884,Large (2000-5000)
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,53.539172,Large (2000-5000)
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,90.540541,Small (<1000)


In [33]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean(numeric_only = True)["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean(numeric_only = True)["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean(numeric_only = True)["%Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean(numeric_only = True)["%Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean(numeric_only = True)["%Overall Passing"]

In [34]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.concat([size_math_scores,
                            size_reading_scores,
                            size_passing_math,
                            size_passing_reading,
                            size_overall_passing],
                            axis=1)

# Display results
size_summary

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,89.883853
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,90.621535
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,58.286003


In [35]:
# Group schools by school type

charter_school_summary= per_school_summary[(per_school_summary["School Type"] == "Charter")]
district_school_summary= per_school_summary[(per_school_summary["School Type"] == "District")]


## Scores by School Type

In [36]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(per_school_summary["School Type"]).mean(numeric_only = True)
type_reading_scores = per_school_summary.groupby(per_school_summary["School Type"]).mean(numeric_only = True)
type_passing_math = per_school_summary.groupby(per_school_summary["School Type"]).mean(numeric_only = True)
type_passing_reading = per_school_summary.groupby(per_school_summary["School Type"]).mean(numeric_only = True)
type_overall_passing = per_school_summary.groupby(per_school_summary["School Type"]).mean(numeric_only = True)

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["%Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["%Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["%Overall Passing"]

In [37]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.concat([average_math_score_by_type,
                            average_reading_score_by_type,
                            average_percent_passing_math_by_type,
                            average_percent_passing_reading_by_type,
                            average_percent_overall_passing_by_type],
                            axis=1)

# Display results
type_summary

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,90.432244
District,76.956733,80.966636,64.302528,78.324559,53.672208
